In [62]:
!pip install ktrain #installing ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICE']='0'
%reload_ext autoreload
%autoreload 2
%matplotlib inline



In [64]:
import ktrain             # importing ktrain. ktrain: used to train Deep learning model
from ktrain import text   # text provides many functionalities of NLP
from sklearn.datasets import fetch_20newsgroups # importing dataset

In [79]:
categories = [ 'rec.motorcycles', 'rec.sport.baseball','rec.sport.hockey','sci.med', 'soc.religion.christian'] # using subset of  main dataset for classification item in list will only be called from dataset
train=fetch_20newsgroups(                                                              # loading training subset
    subset = 'train',
    categories=categories,
    shuffle = True,
    random_state = 0
)

In [80]:
test = fetch_20newsgroups(          # loading testing subset
    subset = 'test',
    categories = categories,
    shuffle = True,
    random_state=0


)

In [81]:
test.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [82]:
train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [69]:
test.target

array([0, 4, 0, ..., 4, 2, 2])

In [83]:
train.target

array([1, 1, 2, ..., 2, 4, 0])

In [70]:
test.target_names

['rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.med',
 'soc.religion.christian']

In [84]:
train.target_names

['rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.med',
 'soc.religion.christian']

Training & Testing Split


In [85]:
# train test split
x_train = train.data
y_train = train.target

x_test = test.data
y_test = test.target

In [86]:
model_name='distilbert-base-uncased'  # calling pre-trained model
trans= text.Transformer(model_name, maxlen=128, class_names=categories)

In [87]:
train_data = trans.preprocess_train(x_train,y_train)   # preprocessing training data

preprocessing train...
language: en
train sequence lengths:
	mean : 281
	95percentile : 796
	99percentile : 1621


Is Multi-Label? False


In [88]:
test_data = trans.preprocess_test(x_test, y_test)   # preprocessing testing data

preprocessing test...
language: en
test sequence lengths:
	mean : 280
	95percentile : 719
	99percentile : 1860


In [89]:
model= trans.get_classifier()
learner= ktrain.get_learner(model, train_data=train_data, val_data=test_data, batch_size = 16) # Creating Learner object using ktrain

In [90]:
# learner.lr_find(show_plot=True,max_epochs=5) -->to find best learning rate & ploting graph

In [91]:
learner.fit_onecycle(1e-4,2 ) # applying one cycle method for Learning rate



begin training using onecycle policy with max lr of 0.0001...
Epoch 1/2
187/187 [==============================] - 1933s 10s/step - loss: 0.5033 - accuracy: 0.8414 - val_loss: 0.3237 - val_accuracy: 0.9145
Epoch 2/2
187/187 [==============================] - 1969s 11s/step - loss: 0.1069 - accuracy: 0.9726 - val_loss: 0.2097 - val_accuracy: 0.9416


In [92]:
learner.validate(class_names = categories)  # validating precision,recall,accuracy f1-score

63/63 [==============================] - 327s 5s/step
                        precision    recall  f1-score   support

       rec.motorcycles       0.88      0.98      0.93       398
    rec.sport.baseball       0.93      0.92      0.92       397
      rec.sport.hockey       0.97      0.87      0.92       399
               sci.med       0.96      0.96      0.96       396
soc.religion.christian       0.98      0.98      0.98       398

              accuracy                           0.94      1988
             macro avg       0.94      0.94      0.94      1988
          weighted avg       0.94      0.94      0.94      1988



array([[390,   2,   1,   4,   1],
       [ 16, 364,  10,   5,   2],
       [ 23,  24, 347,   5,   0],
       [ 13,   1,   0, 379,   3],
       [  3,   0,   0,   3, 392]])

In [93]:
learner.view_top_losses(n=5,preproc=trans) # shows datapoints from our dataset which were misclassified by model

63/63 [==============================] - 325s 5s/step
----------
id:733 | loss:7.07 | true:rec.sport.baseball | pred:soc.religion.christian)

----------
id:755 | loss:6.51 | true:rec.sport.baseball | pred:sci.med)

----------
id:659 | loss:6.37 | true:rec.sport.baseball | pred:sci.med)

----------
id:1111 | loss:6.37 | true:rec.sport.baseball | pred:rec.sport.hockey)

----------
id:593 | loss:6.21 | true:rec.motorcycles | pred:sci.med)



Sample output's of the model


In [94]:
#loading datapoint using id
x_test[755]

'Organization: University of Notre Dame - Office of Univ. Computing\nFrom: <RVESTERM@vma.cc.nd.edu>\nSubject: Re: USA McWeekly Stats\n <93108.172841RVESTERM@vma.cc.nd.edu>\n <1993Apr20.033504.13966@mnemosyne.cs.du.edu>\nLines: 36\n\nIn article <1993Apr20.033504.13966@mnemosyne.cs.du.edu>, gspira@nyx.cs.du.edu\n(Greg Spira) says:\n>\n><RVESTERM@vma.cc.nd.edu> writes:\n>\n>>In article <franjion.734996049@spot.Colorado.EDU>, franjion@spot.Colorado.EDU\n>>(John Franjione) says:\n>>>\n>>>Also, I have the impression from reading this group and Bill James\n>>>that Elias is a bunch of money-grubbing jerks whose mission is to\n>>>charge as much as they can for baseball statistical info\n>>>\n>\n>>and bill james is not? yeah.  sure.  do you own "the bill james players\n>>rating book"?\n>\n>Uh, Bill James doesn\'t sell statistics.  He sells books with statistics,\n>but he is not in the business of providing stats like Elias, STATS,\n>Howe, Baseball workshop etc. are.\n>\n>Greg\n\nfunny, it seems 

In [102]:
predictor = ktrain.get_predictor(learner.model,preproc=trans)     
test1 = input("Enter something to classify categories of news :")  #true = sci.med,lets's check what model predict
predictor.predict(test1)

Enter something to classify categories of news :news Chronic persistent hepatitis is usually diagnosed when someone does a liver
1/1 [==============================] - 0s 94ms/step


'sci.med'

In [98]:
predictor = ktrain.get_predictor(learner.model,preproc=trans)   
test2 ="You forgot one thing All have sinned and fallen short of the glory of God"    # true = religion.christian ,lets's check what model predict
predictor.predict(test2)  

1/1 [==============================] - 0s 138ms/step


'soc.religion.christian'

In [99]:
predictor = ktrain.get_predictor(learner.model,preproc=trans)    
test3 =" Is there a pricing guide for new/used motorcycles (Blue Book)?"   # true = motorcycle,lets's check what model predict
predictor.predict(test3)

1/1 [==============================] - 0s 138ms/step


'rec.motorcycles'

In [100]:
predictor = ktrain.get_predictor(learner.model,preproc=trans)    
test4 ="The hard work paid off late in the second, when Tie Domi caught Vancouver goalie Kirk McLean by surprise with a sharp angle shot to make the score 2-1. "   # true = hockey,lets's check what model predict
predictor.predict(test4)

1/1 [==============================] - 0s 137ms/step


'rec.sport.hockey'

In [101]:
predictor = ktrain.get_predictor(learner.model,preproc=trans)   
test5="My brother purchased baseball tickets for Texas Rangers vs Toronto Bluejays in July, but he was unable to get vacation days to get there."    #  true  = baseball ,lets's check what model predict
predictor.predict(test5)

1/1 [==============================] - 0s 179ms/step


'rec.sport.baseball'